# PITN - T1w Image Downsampling and Processing

Code by:

Tyler Spears - tas6hh@virginia.edu

Dr. Tom Fletcher

## Imports & Environment Setup

In [ ]:
# Automatically re-import project-specific modules.
%load_ext autoreload
%autoreload 1

# imports
import collections
import functools
import io
import math
import itertools
import os
import shutil
import pathlib
import inspect
import random
import subprocess
import sys
import warnings
from pathlib import Path
import typing

import ants
import dipy
import dipy.core
import dipy.reconst
import dipy.reconst.dti
import dipy.segment.mask
import dipy.viz
import dipy.viz.regtools
import dotenv

# visualization libraries
%matplotlib inline
import matplotlib as mpl
import mpl_toolkits
import matplotlib.pyplot as plt
import seaborn as sns

import IPython

# Try importing GPUtil for printing GPU specs.
# May not be installed if using CPU only.
try:
    import GPUtil
except ImportError:
    warnings.warn("WARNING: Package GPUtil not found, cannot print GPU specs")
from tabulate import tabulate
from IPython.display import display, Markdown
import ipyplot

# Data management libraries.
import nibabel as nib
import natsort
from natsort import natsorted
import box
from box import Box
import pprint
from pprint import pprint as ppr

# Computation & ML libraries.
import numpy as np
import skimage
import skimage.filters
import torch
import torch.nn.functional as F
import torchio
import monai

plt.rcParams.update({"figure.autolayout": True})
plt.rcParams.update({"figure.facecolor": [1.0, 1.0, 1.0, 1.0]})

# Set print options for ndarrays/tensors.
np.set_printoptions(suppress=True, edgeitems=2, threshold=100, linewidth=88)
torch.set_printoptions(
    sci_mode=False, edgeitems=2, threshold=100, linewidth=88, profile="short"
)

In [ ]:
# Update notebook's environment variables with direnv.
# This requires the python-dotenv package, and direnv be installed on the system
# This will not work on Windows.
# NOTE: This is kind of hacky, and not necessarily safe. Be careful...
# Libraries needed on the python side:
# - os
# - subprocess
# - io
# - dotenv

# Form command to be run in direnv's context. This command will print out
# all environment variables defined in the subprocess/sub-shell.
command = f"direnv exec {os.getcwd()} /usr/bin/env"
# Run command in a new subprocess.
proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, cwd=os.getcwd())
# Store and format the subprocess' output.
proc_out = proc.communicate()[0].strip().decode("utf-8")
# Use python-dotenv to load the environment variables by using the output of
# 'direnv exec ...' as a 'dummy' .env file.
dotenv.load_dotenv(stream=io.StringIO(proc_out), override=True);

In [ ]:
# Project-specific scripts
# It's easier to import it this way rather than make an entirely new package, due to
# conflicts with local packages and anaconda installations.
# You made me do this, poor python package management!!
if "PROJECT_ROOT" in os.environ:
    lib_location = str(Path(os.environ["PROJECT_ROOT"]).resolve())
else:
    lib_location = str(Path("../../").resolve())
if lib_location not in sys.path:
    sys.path.insert(0, lib_location)
import lib as pitn

# Include the top-level lib module along with its submodules.
%aimport lib
# Grab all submodules of lib, not including modules outside of the package.
includes = list(
    filter(
        lambda m: m.startswith("lib."),
        map(lambda x: x[1].__name__, inspect.getmembers(pitn, inspect.ismodule)),
    )
)
# Run aimport magic with constructed includes.
ipy = IPython.get_ipython()
ipy.run_line_magic("aimport", ", ".join(includes))

In [ ]:
# torch setup
# allow for CUDA usage, if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# keep device as the cpu
# device = torch.device('cpu')
print(device)

### Specs Recording

In [ ]:
%%capture --no-stderr cap
# Capture output and save to log. Needs to be at the *very first* line of the cell.
# Watermark
%load_ext watermark
%watermark --author "Tyler Spears" --updated --iso8601  --python --machine --iversions --githash
if torch.cuda.is_available():

    # GPU information
    # Taken from
    # <https://www.thepythoncode.com/article/get-hardware-system-information-python>.
    # If GPUtil is not installed, skip this step.
    try:
        gpus = GPUtil.getGPUs()
        print("=" * 50, "GPU Specs", "=" * 50)
        list_gpus = []
        for gpu in gpus:
            # get the GPU id
            gpu_id = gpu.id
            # name of GPU
            gpu_name = gpu.name
            driver_version = gpu.driver
            cuda_version = torch.version.cuda
            # get total memory
            gpu_total_memory = f"{gpu.memoryTotal}MB"
            gpu_uuid = gpu.uuid
            list_gpus.append(
                (
                    gpu_id,
                    gpu_name,
                    driver_version,
                    cuda_version,
                    gpu_total_memory,
                    gpu_uuid,
                )
            )

        print(
            tabulate(
                list_gpus,
                headers=(
                    "id",
                    "Name",
                    "Driver Version",
                    "CUDA Version",
                    "Total Memory",
                    "uuid",
                ),
            )
        )
    except NameError:
        print("CUDA Version: ", torch.version.cuda)

else:
    print("CUDA not in use, falling back to CPU")

In [ ]:
# cap is defined in an ipython magic command
print(cap)

Author: Tyler Spears

Last updated: 2021-10-06T16:48:16.146694+00:00

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.23.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.4.0-88-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Git hash: 5123101df9112c32921025ab162ce3c3ee751aa1

matplotlib: 3.4.1
monai     : 0.7.dev2138
numpy     : 1.20.2
torch     : 1.9.0
ipywidgets: 7.6.3
natsort   : 7.1.1
addict    : 2.4.0
box       : 5.4.1
dipy      : 1.4.1
nibabel   : 3.2.1
json      : 2.0.9
GPUtil    : 1.4.0
sys       : 3.8.8 (default, Feb 24 2021, 21:46:12) 
[GCC 7.3.0]
pandas    : 1.2.3
IPython   : 7.23.1
ants      : 0.2.7
torchio   : 0.18.37
seaborn   : 0.11.1

================================================== GPU Specs ==================================================
  id  Name       Driver Version      CUDA Version  Total Memory    uuid
----  ---------  ----------------  --------------  --------------  

### Data Variables & Definitions Setup

In [ ]:
# Set up directories
data_dir = pathlib.Path(os.environ["DATA_DIR"])
processed_data_dir = pathlib.Path(os.environ["WRITE_DATA_DIR"])
hcp_source_data_dir = data_dir / "hcp"
clinic_source_data_dir = processed_data_dir / "oasis3/derivatives/freesurfer"
assert hcp_source_data_dir.exists() and clinic_source_data_dir.exists()

hcp_processed_data_dir = processed_data_dir / "hcp/derivatives/mean-downsample"
clinic_processed_data_dir = processed_data_dir / "oasis3/derivatives/mean-downsample"
assert hcp_processed_data_dir.exists() and clinic_processed_data_dir.exists()

In [ ]:
hcp_spacing = 0.7
clinic_spacing = 1.0
downscale_factor = clinic_spacing / hcp_spacing

# Set the output directory based on the target voxel size.
hcp_processed_data_dir /= f"scale-{clinic_spacing:.2f}mm"
clinic_processed_data_dir /= "scale-orig"
assert hcp_processed_data_dir.exists() and clinic_processed_data_dir.exists()

## Load & Process Data

### HCP Data

In [ ]:
hcp_subj_data: dict = dict()
source_file_relative_dir = "T1w/"
source_filename = "T1w_acpc_dc_restore_brain.nii.gz"
mask_file_relative_dir = "T1w/"
mask_filename = "brainmask_fs.nii.gz"

downscaler = pitn.transforms.FractionalMeanDownsampleTransform(
    hcp_spacing, clinic_spacing
)

for hcp_subj_dir in hcp_source_data_dir.glob("[0-9]*"):
    subj_id = hcp_subj_dir.name
    print(subj_id)

    source_file = (hcp_subj_dir / source_file_relative_dir) / source_filename
    mask_file = (hcp_subj_dir / mask_file_relative_dir) / mask_filename

    source_img = torchio.ScalarImage(source_file)
    mask = torchio.LabelMap(mask_file)

    # Mean-downscale image and mask to the target spatial resolution.
    #     breakpoint()
    downscale_source_img = downscaler(source_img)
    downscale_mask = downscaler(mask)

    # Construct NIFTI objects from the original file's header information, the new
    # downsampled data, and the new downsampled data's affine matrix.
    nifti_header = nib.Nifti1Image.from_filename(source_file).header
    nifti_source_img = nib.Nifti1Image(
        downscale_source_img.data.cpu().numpy(),
        downscale_source_img.affine,
        header=nifti_header,
    )
    nifti_mask = nib.Nifti1Image(
        downscale_mask.data.cpu().numpy(), downscale_mask.affine, header=nifti_header
    )

    write_subj_dir = hcp_processed_data_dir / f"sub-{subj_id}"
    img_dir = write_subj_dir / "anat"
    img_dir.mkdir(parents=True, exist_ok=True)
    nib.save(
        nifti_source_img,
        img_dir
        / f"sub-{subj_id}_meandownsample-{clinic_spacing:.2f}mm_{source_filename}",
    )

    mask_dir = write_subj_dir / "mask"
    mask_dir.mkdir(parents=True, exist_ok=True)
    nib.save(
        nifti_mask,
        mask_dir
        / f"sub-{subj_id}_meandownsample-{clinic_spacing:.2f}mm_{mask_filename}",
    )

    hcp_subj_data[subj_id] = {"t1": downscale_source_img, "mask": downscale_mask}
#     f1 = dipy.viz.regtools.plot_slices(mask.data[0].cpu().numpy())
#     f1.set_dpi(120)
#     f1.show()
#     f2 = dipy.viz.regtools.plot_slices(downscale_source_img.data[0].cpu().numpy())
#     f2.set_dpi(120)
#     f2.show()

---

### Clinical Data

#### Generate Mask Images from Freesurfer T1w Output

In [ ]:
clinic_subj_data: dict = dict()
source_file_relative_dir = "mri/"
source_filename = "brain.finalsurfs.mgz"

for clinic_subj_dir in clinic_source_data_dir.glob("OAS[0-9]*"):
    subj_id = clinic_subj_dir.name
    print(subj_id)

    source_file = (clinic_subj_dir / source_file_relative_dir) / source_filename

    source_img = nib.load(source_file)
    source_img = nib.Nifti1Image.from_image(source_img)

    mask = nib.load(source_file)
    mask = nib.Nifti1Image.from_image(mask)
    mask_data = mask.get_fdata()
    thresh = 1e-6
    mask_data = mask_data >= thresh
    volume_thresh = np.round(4 / 3 * (np.pi * 4) ** 3).astype(int)
    mask_data = skimage.morphology.remove_small_holes(mask_data, volume_thresh)
    mask_data = skimage.morphology.binary_opening(
        mask_data, skimage.morphology.ball(5, dtype=bool)
    )
    mask_data = skimage.morphology.remove_small_objects(mask_data)

    mask = nib.Nifti1Image(mask_data.astype(np.uint8), affine=None, header=mask.header)

    write_subj_dir = clinic_processed_data_dir / f"sub-{subj_id}"
    img_dir = write_subj_dir / "anat"
    img_dir.mkdir(parents=True, exist_ok=True)
    nib.save(
        source_img,
        img_dir / f"sub-{subj_id}_{source_file.stem}_T1w.nii.gz",
    )

    mask_dir = write_subj_dir / "mask"
    mask_dir.mkdir(parents=True, exist_ok=True)
    nib.save(
        mask,
        mask_dir / f"sub-{subj_id}_mask.nii.gz",
    )

In [ ]:
f1 = dipy.viz.regtools.plot_slices(mask_data)
f1.set_dpi(150)
f1.show()